In [ ]:
import pandas as pd

pd.set_option("display.max_columns", None)

In [ ]:
table_path = "../raw_docs/done_EMBA-35. Exit Survey.xlsx"
raw_data = pd.read_excel(table_path, skiprows=[0, 2, 3], sheet_name="Raw data")
processed_data = pd.read_excel(table_path, skiprows=[0, 2, 3], sheet_name="edited")

In [ ]:
processed_data.head(3)

#### Converting table to a desired structure

In [ ]:
service_cols = [col for col in raw_data.columns if col.startswith("S")]
unnamed_cols = [col for col in raw_data.columns if col.startswith("Unnamed:")]
cols_to_drop = ["PS0 Start", "PD0 Duration , sec"] + service_cols + unnamed_cols

In [ ]:
raw_data = raw_data.drop(columns=cols_to_drop)
processed_data = processed_data.drop(columns=cols_to_drop)

In [ ]:
# drop all rows where participant name is None
name_col = "Q16  - 🔴 Укажите  фамилию и имя"
raw_data = raw_data.dropna(subset=[name_col], ignore_index=True)

In [ ]:
# replace professors names with their standard form (block Q6)
q6_cols = [col for col in raw_data.columns if col.startswith("Q6")]
raw_data[q6_cols] = processed_data[q6_cols]

#### Filling Nans

In [ ]:
raw_data.isna().sum()

In [ ]:
raw_data[q6_cols] = raw_data[q6_cols].fillna(
    0
)  # if a student doesnt remember any professors

age_col = "Q15 🔴  Укажите ваш  возраст"
raw_data[age_col] = raw_data[age_col].fillna(
    raw_data[age_col].median()
)  # if the age was not provided fill with mode

score_cols = [
    col
    for col in raw_data.columns
    if col.startswith("Q7") or col.startswith("Q8") or col.startswith("Q9")
]
for col in score_cols:
    raw_data[col] = raw_data[col].fillna(
        int(raw_data[col].mean())
    )  # if a value in scoring col is missing, replace by the mean

free_answer_cols = [
    "Q4.1 Каковы, на ваш взгляд,  сильные и слабые стороны программы? - Сильные стороны",
    "Q4.2 Каковы, на ваш взгляд,  сильные и слабые стороны программы? - Слабые стороны",
    "Q10 Оставьте ваши  пожелания и дополнительные комментарии  по программе",
    "Q10 Оставьте ваши  пожелания и дополнительные комментарии  по программе",
    "Q11.1.O В качестве приглашенного спикера - брендинг, корпфин",
    "Q11.5.O Другое - протагонист кейса",
    "Q13.6.O Другое - Other",
    "Q14.20.O Другое - Other",
]
for col in free_answer_cols:
    raw_data[col] = raw_data[col].fillna(
        "No comments"
    )  # set to "No comments" if the answer wasnt provided in cols with free answer

raw_data.isna().any().any()

#### Save the results

In [ ]:
raw_data.head(3)

In [ ]:
raw_data.to_excel("../artifacts/processed.xlsx", index=False)